Some general notes:
* `product_sku` in the `products` table is analogous to `product_id` in the `orders` table.
* `cust_no` in the `orders` table is analogous to `customer_number` in the `customers` table.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
import requests
%reload_ext sql

In [2]:
customers = pd.read_csv(r"customers.csv")
orders = pd.read_csv(r"orders.csv", parse_dates=['ts'])
products = pd.read_csv(r"products.csv", error_bad_lines=False)

In [3]:
engine = create_engine('sqlite:///coolbusinessllcdb.sql', echo=False)
customers.to_sql('customers', con=engine, if_exists='replace')
orders.to_sql('orders', con=engine, if_exists='replace')
products.to_sql('products', con=engine, if_exists='replace')
os.environ["DATABASE_URL"]='sqlite:///coolbusinessllcdb.sql'

In [4]:
%%sql
UPDATE products
SET msrp = "Unnamd: 3"
WHERE "Unnamed: 3" IS NOT NULL

2 rows affected.


[]

1. How many unique customers have had any transaction with Cool Business in 2021? 

In [5]:
%%sql 
SELECT COUNT(DISTINCT cust_no)
FROM orders
WHERE strftime('%Y',ts) = '2021'

 * sqlite:///coolbusinessllcdb.sql
Done.


COUNT(DISTINCT cust_no)
48


2. What are the last names of the customers who purchased tyx/q/109 in February 2021?

In [6]:
%%sql 
SELECT DISTINCT customers.last_name
FROM orders
LEFT JOIN customers
ON orders.cust_no = customers.customer_number
WHERE orders.product_id = 'tyx/q/109' AND strftime('%Y-%m',ts) = '2021-02'

 * sqlite:///coolbusinessllcdb.sql
Done.


last_name
Clarkson
Dube
Metz
Oneal
Haywood


3. How many total `orders` (including *ALL* `transact_types`) were made for fff/x/990?

In [7]:
%%sql 
SELECT COUNT(orders.order_number) AS num_orders
FROM orders
WHERE orders.product_id = 'fff/x/990'

 * sqlite:///coolbusinessllcdb.sql
Done.


num_orders
9


4. How many `orders` with a `transact_type` of `SALE` were to customers in the state of New Jersey? The state code `NJ` is in the `address` field in the `customers` table.

In [8]:
%%sql
SELECT COUNT(DISTINCT orders.order_number)
FROM orders
LEFT JOIN customers
ON orders.cust_no = customers.customer_number
WHERE customers.address LIKE '%NJ%' AND orders.transact_type = 'SALE'

 * sqlite:///coolbusinessllcdb.sql
Done.


COUNT(DISTINCT orders.order_number)
19


5. How many unique customers have had sucessful, non-returned transactions with Cool Business in 2021? Do not include customers who have *not* had a succesful transaction in 2021. A sucessful transaction is one which is of transact_type `SALE`, and has no subsequent return against the same `order_number`.

In [9]:
%%sql 

SELECT COUNT(DISTINCT cust_no)
FROM orders
WHERE (strftime('%Y', ts) = '2021') AND (order_number NOT IN (
    SELECT order_number
    FROM orders
    WHERE transact_type IN ('VOID', 'FAILED')))


 * sqlite:///coolbusinessllcdb.sql
Done.


COUNT(DISTINCT cust_no)
44


6. Cool Business needs a list of all the customers with a `VOID` order for any Product ID/SKU that starts with the letter `t`. This list will then be loaded into an email tool for a personalized email campaign. Write a query which returns a table with the purchase dates, customer numbers, and first names of this group. In cases where first name is missing in the `customers` table, the string value `"Customer"` should be substituted. Assume the missing-first-name rows are `NULL` if these CSV files were loaded into a SQL database.

In [10]:
%%sql 
SELECT orders.ts, orders.cust_no, 
    CASE WHEN customers.first_name IS NULL THEN "Customer"
    ELSE customers.first_name END AS first_name
FROM orders
LEFT JOIN customers
ON orders.cust_no = customers.customer_number
WHERE orders.transact_type = 'VOID' AND orders.product_id LIKE 't%'

 * sqlite:///coolbusinessllcdb.sql
Done.


ts,cust_no,first_name
2021-02-25 05:44:54.000000,C1002958,Roxann
2021-01-29 21:19:46.000000,C1002942,Suzanna


7. Generate a count of customers who have made 3 or more purchases.

In [11]:
%%sql
WITH cte AS (
SELECT DISTINCT orders.cust_no, COUNT(orders.order_number) OVER(PARTITION BY orders.cust_no) AS num_purchases
FROM orders)

SELECT COUNT(cust_no) as num_customers
FROM cte
WHERE num_purchases >=3


 * sqlite:///coolbusinessllcdb.sql
Done.


num_customers
15


## City Bikes REST API 

* Calls the City Bikes REST API's (http://api.citybik.es/v2/) `networks` endpoint to retrieve information about bike shares around the world.
* Creates a local CSV file containing 1 row for each network, with the following details only: Name | Company | Country

In [12]:
url = 'http://api.citybik.es/v2/networks'
r = requests.get(url)

result = r.json()
networks_new = []
networks = result.get('networks')
for each in networks:
    each.update(each.get('location'))
    networks_new.append({'Name':each.get('name'), 'Company':each.get('company'), 'Country':each.get('country')})
pd.DataFrame(networks_new).to_csv("./coding/rest_api_output.csv")